# 使用 Cognee 构建具有持久记忆的 AI 代理

本笔记本演示了如何使用 [**cognee**](https://www.cognee.ai/) 构建具有复杂记忆功能的智能 AI 代理。Cognee 是一种开源的 AI 记忆系统，它结合了知识图谱、语义搜索和会话管理，能够创建具有上下文感知能力的 AI 系统。

## 🎯 学习目标

通过本教程，你将学会：
- **构建基于嵌入的知识图谱**：将非结构化文本转化为结构化、可查询的知识
- **实现会话记忆**：创建多轮对话并自动保留上下文
- **持久化对话**：可选择将重要的交互存储在长期记忆中以供将来参考
- **使用自然语言查询**：在新对话中访问并利用历史上下文
- **可视化记忆**：探索代理知识图谱中的关系


## 🏗️ 您将构建的内容

在本教程中，我们将创建一个具有持久记忆的**编码助手**，其功能包括：

### 1. **知识库构建**
   - 获取开发者的个人资料和专业信息
   - 处理 Python 编程原则和最佳实践
   - 存储开发者与 AI 助手之间的历史对话

### 2. **会话感知对话**
   - 在同一会话中保持上下文关联
   - 自动缓存每个问题/答案对以提高检索效率
   - 基于对话历史提供连贯且有上下文的回复

### 3. **长期记忆**
   - 将重要对话存储到长期记忆中
   - 从知识库和过去的会话中检索相关记忆以辅助新交互
   - 构建一个不断增长的知识库，随着时间推移不断改进

### 4. **智能记忆检索**
   - 使用图感知语义搜索在所有存储的知识中查找相关信息
   - 按数据子组（开发者信息 vs. 编程原则）过滤搜索
   - 结合多个数据源提供全面的答案


## 📋 前置条件与设置

### 系统要求

在开始之前，请确保您具备以下条件：

1. **Python 环境**
   - Python 3.9 或更高版本
   - 推荐使用虚拟环境

2. **Redis 缓存**（用于会话管理，必需）
   - 本地 Redis: `docker run -d -p 6379:6379 redis`
   - 或使用托管的 Redis 服务

3. **LLM API 访问**
   - OpenAI API 密钥或其他提供商（参见[文档](https://docs.cognee.ai/setup-configuration/llm-providers)）

4. **数据库配置**
   - 默认无需配置。Cognee 使用基于文件的数据库（LanceDB 和 Kuzu）
   - 可选：您可以设置 Azure AI Search 作为向量存储（参见[文档](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch)）

### 环境配置

在您的项目目录中创建一个 `.env` 文件，并包含以下变量：

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ 理解Cognee的记忆架构

### Cognee的工作原理

Cognee提供了一个复杂的记忆系统，超越了简单的键值存储：

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### 关键组件：

1. **知识图谱**：存储实体、关系和语义连接
2. **向量嵌入**：支持对所有存储信息的语义搜索
3. **会话缓存**：在会话内及跨会话维持对话上下文
4. **节点集**：将数据组织成逻辑类别以便于目标检索

### 本教程中的记忆类型：

- **持久记忆**：知识图谱中的长期存储
- **会话记忆**：Redis缓存中的临时对话上下文
- **语义记忆**：基于向量的相似性搜索覆盖所有数据


## 📦 安装所需软件包

安装带有 Redis 支持的 Cognee 以进行会话管理：


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 初始化环境并加载库

确保：
1. Redis 正在运行（例如，通过 Docker：`docker run -d -p 6379:6379 redis`）
2. 在导入缓存模块之前设置环境变量
3. 如果需要，重新启动内核并按顺序运行单元格

以下单元格将会：
1. 从 `.env` 文件加载环境变量
2. 使用您的 LLM 设置配置 Cognee
3. 启用会话管理的缓存功能
4. 验证所有组件是否正确连接


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 配置存储目录

Cognee 使用两个独立的目录进行操作：
- **数据根目录**：存储已导入的文档和处理后的数据
- **系统根目录**：包含知识图谱数据库和系统元数据

在本教程中，我们将创建独立的目录，如下所示：


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 重置记忆状态

在我们开始构建记忆系统之前，先确保我们从头开始。

> 💡 **提示**：如果您希望在以后使用此笔记本时保留之前运行的现有记忆，可以跳过此步骤。


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 第1部分：构建知识库

### 我们开发者助手的数据来源

我们将引入三种类型的数据来创建一个全面的知识库：

1. **开发者档案**：个人专业知识和技术背景  
2. **Python最佳实践**：Python之禅及其实用指南  
3. **历史对话**：开发者与AI助手之间的过去问答记录  

这些多样化的数据使我们的代理能够：  
- 理解用户的技术背景  
- 在推荐中应用最佳实践  
- 从以往成功的互动中学习  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 将数据转化为知识图谱

现在我们将把原始文本转化为结构化的记忆。这个过程包括：

1. **将数据添加到 NodeSets**：将信息组织到逻辑分类中
   - `developer_data`：开发者资料和对话
   - `principles_data`：Python最佳实践和指导原则

2. **运行 Cognify Pipeline**：提取实体、关系并创建嵌入
   - 识别关键概念
   - 在相关信息之间创建语义连接
   - 生成向量嵌入

这可能需要一些时间，因为LLM正在处理文本并构建图谱结构：


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 可视化知识图谱

让我们来探索知识图谱的结构。可视化内容包括：
- **节点**：从文本中提取的实体（概念、技术、人物）
- **边**：实体之间的关系和连接
- **聚类**：按语义相似性分组的相关概念

在浏览器中打开生成的 HTML 文件，以交互方式探索图谱：


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 用 Memify 丰富记忆

`memify()` 函数会分析知识图谱并生成关于数据的智能规则。这个过程：
- 识别模式和最佳实践
- 根据内容创建可操作的指导方针
- 建立不同知识领域之间的关系

这些规则帮助代理在回答问题时做出更明智的决策。捕获第二个可视化可以帮助您比较图谱在丰富后如何变得更加密集。


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 第2部分：智能记忆检索

### 示例 1：跨文档知识整合

现在我们的知识图谱已经构建完成，让我们测试一下Cognee如何结合多个来源的信息来回答复杂问题。

第一个查询展示了：
- **语义理解**：即使未明确提及，也能找到相关概念
- **交叉引用**：结合开发者档案与Python原则
- **上下文推理**：将最佳实践应用于具体实现

### 示例 2：使用NodeSets进行筛选搜索

第二个查询展示了如何针对知识图谱的特定子集进行搜索：
- 使用`node_name`参数仅在`principles_data`中搜索
- 从特定知识领域提供集中答案
- 当需要领域特定信息时非常有用


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 第三部分：会话管理设置

### 启用对话记忆

会话管理对于在多次交互中保持上下文至关重要。在这里我们将：

1. **初始化用户上下文**：创建或检索用户档案以进行会话跟踪
2. **配置缓存引擎**：连接到 Redis 以存储对话历史记录
3. **启用会话变量**：设置在查询之间持久化的上下文变量

> ⚠️ **重要**：这需要 Redis 正在运行，并且环境中设置了 `CACHING=true`


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ 辅助函数：查看会话历史

此工具函数允许我们检查存储在 Redis 中的会话历史记录。它的用途包括：
- 调试会话管理
- 验证对话是否被缓存
- 了解代理可用的上下文


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## 第1节：异步支持实验室——第一个问题

开始 `async-support-lab` 课程，询问适用于大规模网页抓取的支持遥测的 asyncio 模式。图谱已经了解 asyncio、aiohttp 和监控实践，因此回答应反映之前的讨论，同时针对新问题进行调整。


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## 检查会话 1 在第一次交互后的内存

在初始问题后立即运行 `show_history(session_1)` 可以确认 Cognee 已将提示和完成内容都写入了 Redis。你应该会看到一个包含并发指导的条目。


In [ ]:
await show_history(session_1)

## 第1节：数据模型的后续讨论

接下来我们问：“什么时候应该选择 dataclasses 而不是 Pydantic？”使用相同的会话 ID。Cognee 应该结合 Python 原则以及之前关于 FastAPI 的讨论，提供细致的建议——展示在命名会话中上下文是可以延续的。


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## 确认会话 1 的历史记录包含两个回合

再次调用 `show_history(session_1)` 应该会显示两个问答条目。这与 Mem0 实验室的“记忆重播”步骤相符，并证明了额外的回合会扩展相同的对话记录。


In [ ]:
await show_history(session_1)

## 第2节：设计评审线程 — 新会话

为了展示线程之间的隔离，我们启动了 `design-review-session`，并请求关于事件评审的日志指导。尽管底层知识库是相同的，但新的会话 ID 会将记录分开。


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## 审查会话 2 历史记录

`show_history(session_2)` 应仅列出设计审查的提示/响应对。将其与会话 1 进行比较，以突出显示 Cognee 如何在重用共享知识图的同时保持独立的记录。


In [ ]:
await show_history(session_2)

## 总结

恭喜你！你刚刚为你的编码助手添加了一个由 Cognee 提供支持的真正的长期记忆层。

在本教程中，你将原始开发者内容（代码、文档、聊天记录）转化为一个图形 + 向量记忆，使你的代理能够进行搜索、推理并不断改进。

你学到了什么

1. **从原始文本到 AI 记忆**：了解 Cognee 如何摄取非结构化数据，并通过结合向量 + 知识图谱架构将其转化为智能、可搜索的记忆。

2. **通过 memify 丰富图谱**：学习如何超越基本的图谱创建，使用 memify 在现有图谱之上添加推导出的事实和更丰富的关系。

3. **多种搜索策略**：学习如何根据代理的需求，使用不同的搜索类型（图感知问答、RAG 风格补全、洞察、原始片段、代码搜索等）查询记忆。

4. **可视化探索**：学习如何使用图形可视化和 Cognee UI 检查和调试 Cognee 构建的内容，从而直观地了解知识的结构。

5. **会话感知记忆**：学习如何将每次会话的上下文与持久的语义记忆结合起来，使代理能够在多次运行中记住内容，同时避免在用户之间泄露信息。


## 关键要点
1. 基于嵌入的知识图谱记忆

    - **结构化理解**：Cognee结合了向量存储和图存储，使您的数据既可以通过语义搜索，也可以通过关系连接进行检索。Cognee默认使用基于文件的数据库（LanceDB用于向量存储，Kuzu用于图数据库）。

    - **关系感知检索**：答案不仅可以基于“相似文本”，还可以基于实体之间的关系。

    - **动态记忆**：记忆层不断演变、增长，并作为一个连接的图谱保持可查询。

2. 搜索与推理模式
    - **混合检索**：搜索结合了向量相似性、图结构和LLM推理，从原始块查找到基于图的问答。

    - **根据任务选择模式**：当需要自然语言答案时使用补全模式；当代理需要原始上下文或驱动自身推理时使用块/摘要/图模式。

3. 个性化、会话感知的代理
    - **会话上下文 + 长期记忆**：Cognee将短期“线程”上下文与长期用户或组织级记忆分开。

## 实际应用

1. **垂直领域AI代理**

    使用此笔记本中的模式，为基于Cognee的检索和推理核心的领域智能助手提供支持：

- **开发者助手**：代码审查、事件分析和架构助手，能够遍历代码、API、设计文档和工单，作为一个单一的记忆图谱。

- **面向客户的助手**：支持或成功代理，从产品文档、常见问题解答、CRM笔记和过去的工单中提取信息，进行基于图的检索并提供引用答案。

- **内部专家助手**：政策、法律或安全助手，能够基于互相关联的规则、指南和历史决策进行推理，而不是孤立的PDF。

    Cognee明确定位为AI代理的持久、准确记忆，提供一个动态知识图谱，作为代理的后端，取代临时组合的向量存储和自定义图代码。

2. **将数据孤岛统一为一个记忆层**

    同样的方法也可以帮助您在分散的来源之间构建统一的记忆层：

- **从孤岛到一个图谱**：将结构化数据（如数据库）和非结构化数据（如文档、聊天记录）导入一个由嵌入支持的单一图谱，而不是为每个系统创建单独的索引。

- **带引用的跨来源推理**：在所有数据上运行多步推理——通过图谱“连接”日志、指标和文档——并仍然返回有出处的答案。

- **知识中心**：在银行或教育等领域，Cognee已经用于将PDF、内部系统和应用数据统一为一个带向量的知识图谱，使代理能够以精确、引用的上下文回答问题。

## 下一步

您已经实现了核心记忆循环。以下是您可以自行尝试的自然扩展（详见 [Cognee文档](https://docs.cognee.ai/)）：

1. **尝试时间感知**：开启时间认知功能，从文本中提取事件和时间戳。

2. **引入基于本体的推理**：为您的领域定义一个OWL本体。使用Cognee的本体支持，使提取的实体和关系基于该架构，从而提高图谱质量和领域特定答案。

3. **添加反馈循环**：让Cognee根据真实用户反馈调整图边权重，使检索随着时间的推移不断改进，而不是保持静态。

4. **优化个性化和会话行为**：使用用户ID、租户和数据集，为每个人或团队提供共享记忆引擎的专属视图。

5. **扩展到更复杂的代理**：将Cognee接入代理框架，构建共享同一记忆层的多代理系统。*Microsoft Agent Framework x Cognee插件即将推出。*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
